# Simulated Method of Moments

Based on McFadden (*Econometrica*, 1989), the Simulated Method of Moments (SMM) is a method to estimate the parameters of a model by matching the moments of the data with the moments of the model. The moments are calculated by simulating the model many times and averaging the results. The method is particularly useful when the model is too complex to be solved analytically.

The SMM estimator, $\hat{\theta}_{S,T}(W)$, is define as:

$$
\hat{\theta}_{S,T}(W)= argmin_{\theta} \left[ \sum_{t=1}^{T}\left(\mu(x_t) - \frac{1}{S} \sum_{s=1}^{S} \mu(x(u^s_t, \theta)) \right) \right]^{'} W_T^{-1} \left[ \sum_{t=1}^{T}\left(\mu(x_t) - \frac{1}{S} \sum_{s=1}^{S} \mu(x(u^s_t, \theta)) \right) \right]
$$

where:
* $x_t$ are a sequence of $T$ observations of the data
* $\mu(\cdot)$ is a function of the data
* $x(u^s_t, \theta)$ is the model simulated data for the $s$-th simulation
* $W_T$ is a weighting matrix

## SMM with the McCall model

We'll use SMM to estimate two parameters of our job search model. Recall that the McCall model had four parameters:
1. $\theta$, the replacement rate for unemployment benefits
2. $\beta$, the discount factor
3. $\mu$, the wage offer distribution mean
4. $\sigma$, the wage offer distribution standard deviation

We'll assume that $\beta=0.99$ and we'll consider the problem of college graduates, for whom the replacement rate is $\theta=0.0$. We'll estimate the mean and standard deviation of the wage offer distribution.

## The data

For data, we'll use the following moments: the fraction of graduates who are employed after 90 days and the mean wage of those who are employed.  For [graduates from my business school who major in economics, finance, and marketing the numbers are](https://www.sc.edu/study/colleges_schools/moore/talent_recruiting/office_of_career_management/company_recruiter_resources/employment_statistics/index.php):

### Data Moments

|Major| Fraction Employed in 90 Days| Mean Salary |
|---|---|---|
|Economics| 86% | $72,542 |
|Accounting| 90% | $70,223 |
|Marketing| 85% | $63,249 |
|Real Estate| 78% | $70,867 |

## Estimation

We'll use an identify matrix as the weight matrix.  We'll need to estimate $\mu$ and $\sigma$ separately for each major. As such, we'll use [Dask](https://www.dask.org) to parallelize the estimation process.


In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dask
from dask import delayed
from dask.distributed import Client
import scipy.optimize as opt
import scipy.stats as stats
import plotly.express as px

## Step 1: Write a function to solve the McCall model

In [2]:
# Write the function to solve the McCall model
def McCallModel(wages, p, beta=0.99, theta=0.0, max_iter=1000, h_tol=1e-6):
    h = 0  # reservation wage guess
    h_dist = 10
    iter = 0
    avg_wage = np.sum(wages * p)
    b = theta * avg_wage
    continue_wage = wages / (1 - beta)
    while (h_dist > h_tol) & (iter < max_iter):
        h_new = b + beta * np.sum(np.maximum(continue_wage, h) * p)
        h_dist = np.abs(h_new - h).max()
        h = h_new
        iter += 1
    w_bar = (1 - beta) * h
    return w_bar

## Step 2: Write functions to calculate the moments

In [3]:
# Write a function for the 1st moment: fraction accepting offer by 90 days
def frac90(R, wage_offers):
    wait_time = np.argmax(wage_offers > R, axis=0)
    return np.mean(wait_time <= 90)

In [4]:
# Write a function for the 2nd moment: mean accepted offer
def mean_accept(R, wage_offers):
    accepted_offers = np.argmax(wage_offers > R, axis=0)
    return np.mean(wage_offers[accepted_offers, :])

## Step 3: Write a function represent the statistical objective function

This will be the moment error function that we want to minimize.

In [ ]:
# Write the statistical objective function
def moment_condition(data_moments, model_moments, W):
    pct_diff = (model_moments - data_moments) / data_moments  # use pct difference so scale invariant
    dist = pct_diff.T @ W @ pct_diff
    return dist

## Step 4: Write the function you'll pass to the optimizer

This function will take the parameters and data moments and return the moment errors.

In [6]:
# Write the SMM estimation function
def SMM(params, wages, data_moments, W, T=1_000, S=10_000):
    # Unpack params
    mu, sigma = params
    # Get probability of offer for each wage in wages
    p = stats.lognorm.pdf(wages, s=sigma, scale=np.exp(mu))
    p = p / p.sum() # normalize bc wage vector is bounded
    # Solve model for reservation wages
    R = McCallModel(wages, p)
    # Draw wage offers from distribution
    wage_offers = np.random.choice(wages, T * S, p=p)
    wage_offers = wage_offers.reshape(T, S)
    # Compute model moments
    model_moments = np.array([frac90(R, wage_offers), mean_accept(R, wage_offers)])
    # Compute distance between data and model moments
    dist = moment_condition(data_moments, model_moments, W)

    return dist

## Step 5: Write a function to estimate the parameters

This function will take the data moments and return the estimated parameters as well as the model moments at those parameter estimates.

In [7]:
# Main function to run the SMM estimation
def estimate_McCall(data_moments):
    # Set seed
    np.random.seed(1234)
    # Set weighting matrix
    W = np.eye(len(data_moments))
    # Initial guess
    params_init = np.array([np.log(50), 0.5])
    # wage support
    w_min, w_max = 10, 500  # min and max values
    n = 10000 # number of grid points for wages
    wages = np.linspace(w_min, w_max, n)  # use linearly spaced grid over the given support
    # Run optimization
    results = opt.minimize(SMM, params_init, args=(wages, data_moments, W), method='Nelder-Mead')
    # Find model moments with estimated parameters
    mu, sigma = results.x
    # Get probability of offer for each wage in wages
    p = stats.lognorm.pdf(wages, s=sigma, scale=np.exp(mu))
    p = p / p.sum() # normalize bc wage vector is bounded
    # Solve model for reservation wages
    R = McCallModel(wages, p)
    # Draw wage offers from distribution
    T = 1_000
    S = 10_000
    wage_offers = np.random.choice(wages, T * S, p=p)
    wage_offers = wage_offers.reshape(T, S)
    # Compute model moments
    model_moments = np.array([frac90(R, wage_offers), mean_accept(R, wage_offers)])
    # Return results
    return results, model_moments

## Step 6: Do the estimation -- for one major

In [8]:
# Do estimation
data_moments = np.array([0.78, 70_867 / 365])
results, model_moments = estimate_McCall(data_moments)
print("Model moments: ", model_moments)
print("Data moments: ", data_moments)

Model moments:  [  0.9981    193.7131293]
Data moments:  [  0.78       194.15616438]


## Step 7: Do the estimation for all majors

In this case, you'll want to loop over all the majors and estimate the parameters for each one.  

Note that we can use `dask.delayed` to parallelize the estimation process since each major is estimated independently.

In [9]:
# Now let's do this for each major.  We can estimate each problem separately.
# Set up the dictionary of data moments:
data_moments_dict = {
    "Economics": np.array([0.86, 72_542 / 365]),
    "Accounting": np.array([0.90, 70_223 / 365]),
    "Marketing": np.array([0.85, 63_249 / 365]),
    "Real Estate": np.array([0.78, 70_867 / 365])
}
# Set up a local cluster
client = Client()
estimation_results_dict = {}
# Loop over each major
lazy_results = []  # set up a list to store the delayed objects
for k, v in data_moments_dict.items():
    # use dask delayed
    lazy_result = delayed(estimate_McCall)(v)
    lazy_results.append(lazy_result)

# Compute the results
results = dask.compute(*lazy_results)
for i, result in enumerate(results):
    k = list(data_moments_dict.keys())[i]
    estimation_results_dict[k] = {
        "mu": result[0].x[0],
        "sigma": result[0].x[1],
        "distance": result[0].fun,
        "model_moments": result[1],
        "data_moments": data_moments_dict[k]
    }

In [10]:
# put results in a dataframe
results_df = pd.DataFrame(estimation_results_dict).T

In [11]:
# Plot mu, by major
# put mean into dollars per year
results_df["mu"] = np.exp(results_df["mu"].astype(float)) * 365
px.bar(results_df, x=results_df.index, y="mu", title="Estimated Mean Salary Offer, by Major")

In [12]:
# Plot how std dev of salary offers varies by major
px.bar(results_df, x=results_df.index, y="sigma", title="Estimated Std Dev of Salary Offers, by Major")

In [13]:
# Compare model to data moments in table
results_df[["model_moments", "data_moments"]]

,model_moments,data_moments
Economics,"[0.9984, 199.71948275521552]","[0.86, 198.74520547945207]"
Accounting,"[0.9989, 191.99496709997]","[0.9, 192.39178082191782]"
Marketing,"[0.9934, 169.73735935672565]","[0.85, 173.28493150684932]"
Real Estate,"[0.9981, 193.7131293009301]","[0.78, 194.15616438356165]"
